In [1]:
import pandas as pd
import re,json
import requests
from pandas import Series,DataFrame

# 读取文件

In [2]:
with open('2017-07.txt','r',encoding = 'utf-8') as file:
    data = file.readlines()
df = pd.DataFrame(data,columns = ['val'])
df[:25]

,val
0,﻿君问数码官方旗舰店:莫辛\n
1,\n
2,==============================================...
3,单聊 ...
4,==============================================...
5,\n
6,\n
7,----------------------------张旭东19900613-------...
8,张旭东19900613(2017-07-28 16:48:17): 我买的那个手机不是送1...
9,君问数码官方旗舰店:冬青(2017-07-28 16:48:17): 欢迎光临君问数码官方...


# 筛选名字和时间

In [3]:
df['name'] = df.val.str.findall(r'^(.*?)\(2017-.*?\)').str[0]
df['time'] = df.val.str.findall(r'(2017-.*?)\): ').str[0]
df.head(15)

,val,name,time
0,﻿君问数码官方旗舰店:莫辛\n,NaN,NaN
1,\n,NaN,NaN
2,==============================================...,NaN,NaN
3,单聊 ...,NaN,NaN
4,==============================================...,NaN,NaN
5,\n,NaN,NaN
6,\n,NaN,NaN
7,----------------------------张旭东19900613-------...,NaN,NaN
8,张旭东19900613(2017-07-28 16:48:17): 我买的那个手机不是送1...,张旭东19900613,2017-07-28 16:48:17
9,君问数码官方旗舰店:冬青(2017-07-28 16:48:17): 欢迎光临君问数码官方...,君问数码官方旗舰店:冬青,2017-07-28 16:48:17


# 能辨别说话人的数据打标签

In [4]:
cond_1 = df.name.notnull()
num_1 = len(df.loc[cond_1])
num_1

14835

In [5]:
df.loc[cond_1,'No'] = list(range(num_1))
df.loc[0,'No'] = -1

# 统计对话数量并标记序号

In [6]:
cond_2 = df.val.str.contains(r'^----------------------------')
num_2 = len(df.loc[cond_2])
num_2

205

In [7]:
df.loc[cond_2,'customer_id'] = ['customer_%s' % i for i in range(205)]
df.loc[cond_2,'customer'] = df.loc[cond_2,'val'].str.findall('-+(.*?)-+$').str[0]
df.head(20)

,val,name,time,No,customer_id,customer
0,﻿君问数码官方旗舰店:莫辛\n,NaN,NaN,-1.0,NaN,NaN
1,\n,NaN,NaN,NaN,NaN,NaN
2,==============================================...,NaN,NaN,NaN,NaN,NaN
3,单聊 ...,NaN,NaN,NaN,NaN,NaN
4,==============================================...,NaN,NaN,NaN,NaN,NaN
5,\n,NaN,NaN,NaN,NaN,NaN
6,\n,NaN,NaN,NaN,NaN,NaN
7,----------------------------张旭东19900613-------...,NaN,NaN,NaN,customer_0,张旭东19900613
8,张旭东19900613(2017-07-28 16:48:17): 我买的那个手机不是送1...,张旭东19900613,2017-07-28 16:48:17,0.0,NaN,NaN
9,君问数码官方旗舰店:冬青(2017-07-28 16:48:17): 欢迎光临君问数码官方...,君问数码官方旗舰店:冬青,2017-07-28 16:48:17,1.0,NaN,NaN


# 给无法辨别说话人的话，找到主人

In [8]:
df.No = df.No.fillna(method = 'ffill')
df.customer = df.customer.fillna(method = 'ffill')
df.customer_id = df.customer_id.fillna(method = 'ffill')
df.tail(10)

,val,name,time,No,customer_id,customer
19594,秋千上的爱情2(2017-07-02 18:51:55): [未知]\n,秋千上的爱情2,2017-07-02 18:51:55,14826.0,customer_204,秋千上的爱情2
19595,君问数码官方旗舰店:莫兰(2017-07-02 18:51:55): 亲，欢迎光临哦，请问...,君问数码官方旗舰店:莫兰,2017-07-02 18:51:55,14827.0,customer_204,秋千上的爱情2
19596,秋千上的爱情2(2017-07-02 18:52:25): 我给你邮过去用其他快递可以吗\n,秋千上的爱情2,2017-07-02 18:52:25,14828.0,customer_204,秋千上的爱情2
19597,秋千上的爱情2(2017-07-02 18:53:33): 有人？？\n,秋千上的爱情2,2017-07-02 18:53:33,14829.0,customer_204,秋千上的爱情2
19598,君问数码官方旗舰店:莫兰(2017-07-02 18:53:41): 亲这里是一对多服务哦...,君问数码官方旗舰店:莫兰,2017-07-02 18:53:41,14830.0,customer_204,秋千上的爱情2
19599,咨询人数有点多麻烦亲耐心等待· 别误会不理亲哦！ 旺旺在线就有客服值班会尽快回复您的哦~\n,NaN,NaN,14830.0,customer_204,秋千上的爱情2
19600,君问数码官方旗舰店:莫兰(2017-07-02 18:53:47): 可以的 亲\n,君问数码官方旗舰店:莫兰,2017-07-02 18:53:47,14831.0,customer_204,秋千上的爱情2
19601,君问数码官方旗舰店:莫兰(2017-07-02 18:53:52): 除了邮政都能收到 亲\n,君问数码官方旗舰店:莫兰,2017-07-02 18:53:52,14832.0,customer_204,秋千上的爱情2
19602,秋千上的爱情2(2017-07-02 18:54:05): 好的\n,秋千上的爱情2,2017-07-02 18:54:05,14833.0,customer_204,秋千上的爱情2
19603,君问数码官方旗舰店:莫兰(2017-07-02 18:55:29): 嗯呢,君问数码官方旗舰店:莫兰,2017-07-02 18:55:29,14834.0,customer_204,秋千上的爱情2


In [9]:
df_customer = df[['No','customer_id','customer']].copy()
df_customer.drop_duplicates(inplace = True)
df_customer.tail(10)

,No,customer_id,customer
19593,14825.0,customer_204,秋千上的爱情2
19594,14826.0,customer_204,秋千上的爱情2
19595,14827.0,customer_204,秋千上的爱情2
19596,14828.0,customer_204,秋千上的爱情2
19597,14829.0,customer_204,秋千上的爱情2
19598,14830.0,customer_204,秋千上的爱情2
19600,14831.0,customer_204,秋千上的爱情2
19601,14832.0,customer_204,秋千上的爱情2
19602,14833.0,customer_204,秋千上的爱情2
19603,14834.0,customer_204,秋千上的爱情2


# 同一个人的一条信息内容合并在一起

In [10]:
grouped = df.groupby('No')['val']
dfn = grouped.agg(lambda x:'\n'.join(x))
dfn = dfn.reset_index()
dfn.tail(10)

,No,val
14826,14825.0,君问数码官方旗舰店:樱花(2017-07-02 18:51:54): 亲亲，不要着急，这边...
14827,14826.0,秋千上的爱情2(2017-07-02 18:51:55): [未知]\n
14828,14827.0,君问数码官方旗舰店:莫兰(2017-07-02 18:51:55): 亲，欢迎光临哦，请问...
14829,14828.0,秋千上的爱情2(2017-07-02 18:52:25): 我给你邮过去用其他快递可以吗\n
14830,14829.0,秋千上的爱情2(2017-07-02 18:53:33): 有人？？\n
14831,14830.0,君问数码官方旗舰店:莫兰(2017-07-02 18:53:41): 亲这里是一对多服务哦...
14832,14831.0,君问数码官方旗舰店:莫兰(2017-07-02 18:53:47): 可以的 亲\n
14833,14832.0,君问数码官方旗舰店:莫兰(2017-07-02 18:53:52): 除了邮政都能收到 亲\n
14834,14833.0,秋千上的爱情2(2017-07-02 18:54:05): 好的\n
14835,14834.0,君问数码官方旗舰店:莫兰(2017-07-02 18:55:29): 嗯呢


In [11]:
dfn.No = dfn.No.astype(str)

In [12]:
grouped = df.groupby(['customer_id','customer'])
grouped.size()

customer_id   customer            
customer_0    张旭东19900613              67
customer_1    陈思涵csh莫颜1314            228
customer_10   tb60557412               53
customer_100  夏涵铄                      12
customer_101  tb6662543_99             35
customer_102  zl811911                 46
customer_103  注册沉睡账户                   53
customer_104  jiangxiaojie122         119
customer_105  以茉                       34
customer_106  realme20000              30
customer_107  t_1486442619962_0722     17
customer_108  洁s雅nuo                  112
customer_109  ww520198399             101
customer_11   15132791532bin1         274
customer_110  tb4777634_2012          159
customer_111  画熏骨迅迅                    64
customer_112  scaoa3                   21
customer_113  a13736769220             32
customer_114  pool018                  56
customer_115  qq195813531              19
customer_116  t_1498910980795_0645    106
customer_117  zly9608                  81
customer_118  tb0934448               246

# 再分字段

In [13]:
dfn['name'] = dfn.val.str.findall(r'^(.*?)\(2017-.*?\)').str[0]
dfn['time'] = dfn.val.str.findall(r'(2017-.*?)\): ').str[0]
cond_3 = dfn.name.notnull()
dfn.loc[cond_3, 'content'] = dfn.loc[cond_3, 'val'].str.split('\(2017-.*?\): ').str[-1]
dfn.content = dfn.content.str.replace('\n', '\t')
dfn.head()

,No,val,name,time,content
0,-1.0,﻿君问数码官方旗舰店:莫辛\n\n\n\n=========================...,NaN,NaN,NaN
1,0.0,张旭东19900613(2017-07-28 16:48:17): 我买的那个手机不是送1...,张旭东19900613,2017-07-28 16:48:17,我买的那个手机不是送100话费吗\t
2,1.0,君问数码官方旗舰店:冬青(2017-07-28 16:48:17): 欢迎光临君问数码官方...,君问数码官方旗舰店:冬青,2017-07-28 16:48:17,欢迎光临君问数码官方旗舰店，我是客服MM-沫沫，很高兴为您服务！天猫玩物节活动将在7月27...
3,2.0,张旭东19900613(2017-07-28 16:48:17): [卡片]\n,张旭东19900613,2017-07-28 16:48:17,[卡片]\t
4,3.0,张旭东19900613(2017-07-28 16:51:07): ？\n,张旭东19900613,2017-07-28 16:51:07,？\t


In [14]:
df_customer.No = df_customer.No.astype(str)

# 合并

In [15]:
df_output = pd.merge(dfn,df_customer,on = 'No',how = 'left')
df_output.head()

,No,val,name,time,content,customer_id,customer
0,-1.0,﻿君问数码官方旗舰店:莫辛\n\n\n\n=========================...,NaN,NaN,NaN,NaN,NaN
1,-1.0,﻿君问数码官方旗舰店:莫辛\n\n\n\n=========================...,NaN,NaN,NaN,customer_0,张旭东19900613
2,0.0,张旭东19900613(2017-07-28 16:48:17): 我买的那个手机不是送1...,张旭东19900613,2017-07-28 16:48:17,我买的那个手机不是送100话费吗\t,customer_0,张旭东19900613
3,1.0,君问数码官方旗舰店:冬青(2017-07-28 16:48:17): 欢迎光临君问数码官方...,君问数码官方旗舰店:冬青,2017-07-28 16:48:17,欢迎光临君问数码官方旗舰店，我是客服MM-沫沫，很高兴为您服务！天猫玩物节活动将在7月27...,customer_0,张旭东19900613
4,2.0,张旭东19900613(2017-07-28 16:48:17): [卡片]\n,张旭东19900613,2017-07-28 16:48:17,[卡片]\t,customer_0,张旭东19900613


In [16]:
del df_output['val']

In [17]:
df_output.head(10)

,No,name,time,content,customer_id,customer
0,-1.0,NaN,NaN,NaN,NaN,NaN
1,-1.0,NaN,NaN,NaN,customer_0,张旭东19900613
2,0.0,张旭东19900613,2017-07-28 16:48:17,我买的那个手机不是送100话费吗\t,customer_0,张旭东19900613
3,1.0,君问数码官方旗舰店:冬青,2017-07-28 16:48:17,欢迎光临君问数码官方旗舰店，我是客服MM-沫沫，很高兴为您服务！天猫玩物节活动将在7月27...,customer_0,张旭东19900613
4,2.0,张旭东19900613,2017-07-28 16:48:17,[卡片]\t,customer_0,张旭东19900613
5,3.0,张旭东19900613,2017-07-28 16:51:07,？\t,customer_0,张旭东19900613
6,4.0,张旭东19900613,2017-07-28 16:51:14,http://item.taobao.com/item.htm?id=5504785197...,customer_0,张旭东19900613
7,5.0,君问数码官方旗舰店:冬青,2017-07-28 16:51:52,亲亲，您好，有什么可以帮助您的吗？[表情]\t,customer_0,张旭东19900613
8,6.0,君问数码官方旗舰店:冬青,2017-07-28 16:51:53,亲亲，不要着急，这边帮您转接我们售后客服给您核实处理问题哦~\t,customer_0,张旭东19900613
9,7.0,张旭东19900613,2017-07-28 16:51:54,[未知]\t,customer_0,张旭东19900613


In [18]:
df_output.tail(10)

,No,name,time,content,customer_id,customer
15031,14825.0,君问数码官方旗舰店:樱花,2017-07-02 18:51:54,亲亲，不要着急，这边帮您转接我们售后客服给您核实处理问题哦~\t,customer_204,秋千上的爱情2
15032,14826.0,秋千上的爱情2,2017-07-02 18:51:55,[未知]\t,customer_204,秋千上的爱情2
15033,14827.0,君问数码官方旗舰店:莫兰,2017-07-02 18:51:55,亲，欢迎光临哦，请问有什么可以帮助您吗？君问数码官方旗舰店-莫兰为您服务。(在线客服是一对...,customer_204,秋千上的爱情2
15034,14828.0,秋千上的爱情2,2017-07-02 18:52:25,我给你邮过去用其他快递可以吗\t,customer_204,秋千上的爱情2
15035,14829.0,秋千上的爱情2,2017-07-02 18:53:33,有人？？\t,customer_204,秋千上的爱情2
15036,14830.0,君问数码官方旗舰店:莫兰,2017-07-02 18:53:41,亲这里是一对多服务哦` \t\t咨询人数有点多麻烦亲耐心等待· 别误会不理亲哦！ 旺旺在线...,customer_204,秋千上的爱情2
15037,14831.0,君问数码官方旗舰店:莫兰,2017-07-02 18:53:47,可以的 亲\t,customer_204,秋千上的爱情2
15038,14832.0,君问数码官方旗舰店:莫兰,2017-07-02 18:53:52,除了邮政都能收到 亲\t,customer_204,秋千上的爱情2
15039,14833.0,秋千上的爱情2,2017-07-02 18:54:05,好的\t,customer_204,秋千上的爱情2
15040,14834.0,君问数码官方旗舰店:莫兰,2017-07-02 18:55:29,嗯呢,customer_204,秋千上的爱情2


In [19]:
df_output = df_output.loc[df_output.name.notnull()].reset_index(drop = True).copy()
df_output

,No,name,time,content,customer_id,customer
0,0.0,张旭东19900613,2017-07-28 16:48:17,我买的那个手机不是送100话费吗\t,customer_0,张旭东19900613
1,1.0,君问数码官方旗舰店:冬青,2017-07-28 16:48:17,欢迎光临君问数码官方旗舰店，我是客服MM-沫沫，很高兴为您服务！天猫玩物节活动将在7月27...,customer_0,张旭东19900613
2,2.0,张旭东19900613,2017-07-28 16:48:17,[卡片]\t,customer_0,张旭东19900613
3,3.0,张旭东19900613,2017-07-28 16:51:07,？\t,customer_0,张旭东19900613
4,4.0,张旭东19900613,2017-07-28 16:51:14,http://item.taobao.com/item.htm?id=5504785197...,customer_0,张旭东19900613
5,5.0,君问数码官方旗舰店:冬青,2017-07-28 16:51:52,亲亲，您好，有什么可以帮助您的吗？[表情]\t,customer_0,张旭东19900613
6,6.0,君问数码官方旗舰店:冬青,2017-07-28 16:51:53,亲亲，不要着急，这边帮您转接我们售后客服给您核实处理问题哦~\t,customer_0,张旭东19900613
7,7.0,张旭东19900613,2017-07-28 16:51:54,[未知]\t,customer_0,张旭东19900613
8,8.0,君问数码官方旗舰店:莫兰,2017-07-28 16:51:54,亲，7月天猫券活动期间咨询量暴增，亲们收到手机后如有操作疑问请留言，可能回复晚些，请您放心...,customer_0,张旭东19900613
9,9.0,君问数码官方旗舰店:莫兰,2017-07-28 16:52:46,亲 您好\t,customer_0,张旭东19900613


In [20]:
cond_4 = df_output.name.str.contains(r'君问数码官方旗舰店')
df_output.loc[cond_4,'Type'] = 2
df_output.loc[~cond_4,'Type'] = 1
df_output.head(10)

,No,name,time,content,customer_id,customer,Type
0,0.0,张旭东19900613,2017-07-28 16:48:17,我买的那个手机不是送100话费吗\t,customer_0,张旭东19900613,1.0
1,1.0,君问数码官方旗舰店:冬青,2017-07-28 16:48:17,欢迎光临君问数码官方旗舰店，我是客服MM-沫沫，很高兴为您服务！天猫玩物节活动将在7月27...,customer_0,张旭东19900613,2.0
2,2.0,张旭东19900613,2017-07-28 16:48:17,[卡片]\t,customer_0,张旭东19900613,1.0
3,3.0,张旭东19900613,2017-07-28 16:51:07,？\t,customer_0,张旭东19900613,1.0
4,4.0,张旭东19900613,2017-07-28 16:51:14,http://item.taobao.com/item.htm?id=5504785197...,customer_0,张旭东19900613,1.0
5,5.0,君问数码官方旗舰店:冬青,2017-07-28 16:51:52,亲亲，您好，有什么可以帮助您的吗？[表情]\t,customer_0,张旭东19900613,2.0
6,6.0,君问数码官方旗舰店:冬青,2017-07-28 16:51:53,亲亲，不要着急，这边帮您转接我们售后客服给您核实处理问题哦~\t,customer_0,张旭东19900613,2.0
7,7.0,张旭东19900613,2017-07-28 16:51:54,[未知]\t,customer_0,张旭东19900613,1.0
8,8.0,君问数码官方旗舰店:莫兰,2017-07-28 16:51:54,亲，7月天猫券活动期间咨询量暴增，亲们收到手机后如有操作疑问请留言，可能回复晚些，请您放心...,customer_0,张旭东19900613,2.0
9,9.0,君问数码官方旗舰店:莫兰,2017-07-28 16:52:46,亲 您好\t,customer_0,张旭东19900613,2.0


In [21]:
df_output.head(100)

,No,name,time,content,customer_id,customer,Type
0,0.0,张旭东19900613,2017-07-28 16:48:17,我买的那个手机不是送100话费吗\t,customer_0,张旭东19900613,1.0
1,1.0,君问数码官方旗舰店:冬青,2017-07-28 16:48:17,欢迎光临君问数码官方旗舰店，我是客服MM-沫沫，很高兴为您服务！天猫玩物节活动将在7月27...,customer_0,张旭东19900613,2.0
2,2.0,张旭东19900613,2017-07-28 16:48:17,[卡片]\t,customer_0,张旭东19900613,1.0
3,3.0,张旭东19900613,2017-07-28 16:51:07,？\t,customer_0,张旭东19900613,1.0
4,4.0,张旭东19900613,2017-07-28 16:51:14,http://item.taobao.com/item.htm?id=5504785197...,customer_0,张旭东19900613,1.0
5,5.0,君问数码官方旗舰店:冬青,2017-07-28 16:51:52,亲亲，您好，有什么可以帮助您的吗？[表情]\t,customer_0,张旭东19900613,2.0
6,6.0,君问数码官方旗舰店:冬青,2017-07-28 16:51:53,亲亲，不要着急，这边帮您转接我们售后客服给您核实处理问题哦~\t,customer_0,张旭东19900613,2.0
7,7.0,张旭东19900613,2017-07-28 16:51:54,[未知]\t,customer_0,张旭东19900613,1.0
8,8.0,君问数码官方旗舰店:莫兰,2017-07-28 16:51:54,亲，7月天猫券活动期间咨询量暴增，亲们收到手机后如有操作疑问请留言，可能回复晚些，请您放心...,customer_0,张旭东19900613,2.0
9,9.0,君问数码官方旗舰店:莫兰,2017-07-28 16:52:46,亲 您好\t,customer_0,张旭东19900613,2.0


In [22]:
df_output.to_csv('wenwen_clean_0814.txt',sep = '\t',index = False,encoding = 'utf-8')